In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')

In [15]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print('Archivo cargado:', filename)


Saving target.csv to target.csv
Saving numerical.csv to numerical.csv
Saving categorical.csv to categorical.csv
Archivo cargado: target.csv
Archivo cargado: numerical.csv
Archivo cargado: categorical.csv


In [17]:
import pandas as pd

target = pd.read_csv('target.csv')
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')


In [18]:
categorical.shape
print(f'The categorical dataframe has {categorical.shape[0]} rows and {categorical.shape[1]} columns')
numerical.shape
print(f'The numerical dataframe has {numerical.shape[0]} rows and {numerical.shape[1]} columns')
target.shape
print(f'The target has {target.shape[0]} rows')

The categorical dataframe has 95412 rows and 22 columns
The numerical dataframe has 95412 rows and 315 columns
The target has 95412 rows


In [20]:
categorical.head()

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3


In [21]:
numerical.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26


In [22]:
target.head()

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [23]:
categorical.isna().sum()

STATE           0
CLUSTER         0
HOMEOWNR        0
GENDER          0
DATASRCE        0
RFA_2R          0
RFA_2A          0
GEOCODE2        0
DOMAIN_A        0
DOMAIN_B        0
ODATEW_YR       0
ODATEW_MM       0
DOB_YR          0
DOB_MM          0
MINRDATE_YR     0
MINRDATE_MM     0
MAXRDATE_YR     0
MAXRDATE_MM     0
LASTDATE_YR     0
LASTDATE_MM     0
FIRSTDATE_YR    0
FIRSTDATE_MM    0
dtype: int64

In [24]:
categorical.dtypes   #it has many numerical values

STATE           object
CLUSTER          int64
HOMEOWNR        object
GENDER          object
DATASRCE         int64
RFA_2R          object
RFA_2A          object
GEOCODE2        object
DOMAIN_A        object
DOMAIN_B         int64
ODATEW_YR        int64
ODATEW_MM        int64
DOB_YR           int64
DOB_MM           int64
MINRDATE_YR      int64
MINRDATE_MM      int64
MAXRDATE_YR      int64
MAXRDATE_MM      int64
LASTDATE_YR      int64
LASTDATE_MM      int64
FIRSTDATE_YR     int64
FIRSTDATE_MM     int64
dtype: object

In [25]:
categorical.select_dtypes(include='number').columns

Index(['CLUSTER', 'DATASRCE', 'DOMAIN_B', 'ODATEW_YR', 'ODATEW_MM', 'DOB_YR',
       'DOB_MM', 'MINRDATE_YR', 'MINRDATE_MM', 'MAXRDATE_YR', 'MAXRDATE_MM',
       'LASTDATE_YR', 'LASTDATE_MM', 'FIRSTDATE_YR', 'FIRSTDATE_MM'],
      dtype='object')

In [26]:
numerical_b = categorical.select_dtypes(include='number')
numerical = pd.merge(numerical, numerical_b, left_index=True, right_index=True)
numerical.shape

(95412, 330)

In [27]:
len(numerical.dtypes) == len(numerical.select_dtypes(include='number').columns)

True

In [28]:
categorical_drop = categorical.select_dtypes(include='number')
categorical_drop.columns

Index(['CLUSTER', 'DATASRCE', 'DOMAIN_B', 'ODATEW_YR', 'ODATEW_MM', 'DOB_YR',
       'DOB_MM', 'MINRDATE_YR', 'MINRDATE_MM', 'MAXRDATE_YR', 'MAXRDATE_MM',
       'LASTDATE_YR', 'LASTDATE_MM', 'FIRSTDATE_YR', 'FIRSTDATE_MM'],
      dtype='object')

In [29]:
categorical = categorical.drop(['CLUSTER', 'DATASRCE', 'DOMAIN_B', 'ODATEW_YR', 'ODATEW_MM', 'DOB_YR', 'DOB_MM', 'MINRDATE_YR', 'MINRDATE_MM', 'MAXRDATE_YR', 'MAXRDATE_MM','LASTDATE_YR', 'LASTDATE_MM', 'FIRSTDATE_YR', 'FIRSTDATE_MM'], axis = 1)
categorical.head()

,STATE,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
0,IL,H,F,L,E,C,T
1,CA,H,M,L,G,A,S
2,NC,U,M,L,E,C,R
3,CA,U,F,L,E,C,R
4,FL,H,F,L,F,A,S


In [30]:
def check_unique_values(df):   #checking unique values in the df
    for col in df:
        print(list(df[col].unique()))

check_unique_values(categorical)

['IL', 'CA', 'NC', 'FL', 'other', 'IN', 'MI', 'MO', 'TX', 'WA', 'WI', 'GA']
['H', 'U']
['F', 'M', 'other']
['L']
['E', 'G', 'F', 'D']
['C', 'A', 'D', 'B']
['T', 'S', 'R', 'U', 'C']


In [31]:
remain_cat = pd.get_dummies(categorical, drop_first = False)
remain_cat.head()

,STATE_CA,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,...,RFA_2A_G,GEOCODE2_A,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_C,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [32]:
len(remain_cat) == len(numerical)

True

In [33]:
transformer = StandardScaler().fit(numerical)
num_standard = transformer.transform(numerical)
num_standard = pd.DataFrame(num_standard, index = numerical.index, columns = numerical.columns.tolist())
num_standard.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,-0.056847,-1.115292e-01,0.509147,0.717411,-0.356881,-0.206977,0.745798,0.284659,-0.826571,0.719013,...,0.370442,3.012879,-0.172698,0.419080,-0.201169,-1.236357,-0.789204,1.257673,-0.714736,1.237697
1,-0.055799,-1.080356e+00,1.097566,0.717411,1.362283,-0.206977,-1.346527,1.675602,-1.221851,-0.191801,...,1.049599,-0.290923,0.200433,0.932504,0.374914,1.266242,-0.789204,1.257673,0.530642,0.985761
2,-0.055799,1.267017e-10,-0.667692,-2.239375,-0.141985,-0.206977,-0.910626,-0.046518,0.020458,-0.191801,...,-1.304814,-0.290923,-0.545828,1.189216,-1.353334,0.014943,-0.789204,1.257673,-0.403392,-1.281661
3,-0.056847,5.804901e-01,-1.844530,-1.130581,-0.141985,-0.206977,-0.649086,-1.040049,-0.092480,-0.874911,...,-0.037053,-0.621303,-2.038350,1.189216,-0.201169,1.015982,-0.789204,1.257673,-1.337425,-1.029725
4,-0.056847,1.134106e+00,-0.667692,-1.869777,6.089983,-0.009563,-0.213185,-1.371225,1.149829,4.362269,...,-0.399270,-0.621303,0.200433,0.932504,0.950996,-1.486617,1.115522,-1.336135,-3.828181,-0.777790


In [34]:
complete = pd.merge(num_standard, remain_cat, left_index=True, right_index=True )
complete.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_G,GEOCODE2_A,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_C,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,-0.056847,-1.115292e-01,0.509147,0.717411,-0.356881,-0.206977,0.745798,0.284659,-0.826571,0.719013,...,0,0,0,1,0,0,0,0,1,0
1,-0.055799,-1.080356e+00,1.097566,0.717411,1.362283,-0.206977,-1.346527,1.675602,-1.221851,-0.191801,...,1,1,0,0,0,0,0,1,0,0
2,-0.055799,1.267017e-10,-0.667692,-2.239375,-0.141985,-0.206977,-0.910626,-0.046518,0.020458,-0.191801,...,0,0,0,1,0,0,1,0,0,0
3,-0.056847,5.804901e-01,-1.844530,-1.130581,-0.141985,-0.206977,-0.649086,-1.040049,-0.092480,-0.874911,...,0,0,0,1,0,0,1,0,0,0
4,-0.056847,1.134106e+00,-0.667692,-1.869777,6.089983,-0.009563,-0.213185,-1.371225,1.149829,4.362269,...,0,1,0,0,0,0,0,1,0,0


In [35]:
complete.shape[0] == target.shape[0]   #checking if the complete df has the same length as target

True

In [36]:
target['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [37]:
X = complete
y = target['TARGET_B']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [38]:
smote = SMOTE()

X_sm, y_sm = smote.fit_resample(X_train, y_train)

y_sm.value_counts()

0    72464
1    72464
Name: TARGET_B, dtype: int64

In [39]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_sm, y_sm)
model.score(X_test, y_test)

0.9484357805376513